# Welcome

Hi! this code shows and explains Debye-Gruneisen model calculations for a given system Energy vs Volume (EV) curve. You will see how the shape of the curve (and the speed of sound) effect the thermodynamic properties of solids. Specifically you will calculate the entropy, heat capcity, and Helmholtz free energy. Step by step explainations will be given so that you can see the inner workings of the Debye-Gruneisen model.

We will predominately use code from the **D**ensity **F**unctional **T**ool**k**it (DFTTK) written by Luke A Myers and the Dr. Nigel Hew. DFTTK can be used to calculate thermodynamic properties from first principles. However, this notebook will focus only on calculations from a given EV curve defined by the user using the four parameter Birch Murnaghan equation of state.


In [1]:
# import the necessary packages 
# should take less than 30 seconds

import dfttk.debye as debye
from dfttk import eos_fit
import numpy as np
import pandas as pd
import plotly.graph_objs as go

KeyboardInterrupt: 

In [ ]:
# volume range in A^3 for the fit
volumes = np.linspace(90, 110, 1001)
temperatures = np.linspace(0, 1000, 101)
atomic_mass = 30 # typically a geometric or log average atomic mass in daltons
gruneisen_x = 1 # small adjustment value 1 for low temperature, 2/3 for high temperature
scaling_factor = 0.617 # scaling factor determined empiraically for FCC metals. Can also be determined from elastic constants
number_of_atoms = 8


In [ ]:
# parameters for the four parameter Birch Murnaghan (BM4) equation of state (EOS) 
v0 = 100 # equilibrium volume in A^3
e0 = 0.0 # energy at v0
b0 = 69 # bulk modulus at v0 in GPa
b0p = 4.14 # pressure derivative of the bulk modulus at v0

eos_parameters = (v0, e0, b0, b0p)


In [ ]:
density = (atomic_mass*number_of_atoms/volumes)
sound_speed = (b0/density)**(1/2)

In [ ]:
# calculate the energy as function of volume from the given parameters
energies = np.zeros(len(volumes))
for volume in volumes:
    energy = eos_fit.murnaghan_equation(volume, v0, e0, b0, b0p)
    energies[np.where(volumes == volume)] = energy

In [1]:
# plot the curve
ev_fig = go.Figure()
ev_fig.add_trace(go.scatter(
    x=volumes,
    y=energies,
    mode='lines'
))

NameError: name 'go' is not defined

In [63]:
# calculate the gruneisen parameter
gamma = debye.gruneisen_parameter(b0p, gruneisen_x)

In [64]:
theta = debye.debye_temperature(
    volumes,
    eos_parameters,
    atomic_mass,
    gamma,
    scaling_factor
)

In [65]:
s_vib_v_t = np.zeros((len(volumes), len(temperatures)))
f_vib_v_t = np.zeros((len(volumes), len(temperatures)))
cv_vib_v_t = np.zeros((len(volumes), len(temperatures)))

for i, volume in enumerate(volumes):
    s_vib = debye.vibrational_entropy(temperatures, theta[i], number_of_atoms)
    f_vib = debye.vibrational_helmholtz_energy(
        temperatures, theta[i], number_of_atoms
    )
    cv_vib = debye.vibrational_heat_capacity(temperatures, theta[i], number_of_atoms)
    s_vib_v_t[i, :] = s_vib
    f_vib_v_t[i, :] = f_vib
    cv_vib_v_t[i, :] = cv_vib

In [66]:
# plot the entropy at volumes 90,95,100,105,110 A^3 as a function of temperature
selected_volumes = [90, 95, 100, 105, 110]
s_fig = go.Figure()
for i, volume in enumerate(selected_volumes):
    s_fig.add_trace(go.Scatter(
        x=temperatures,
        y=s_vib_v_t[np.where(volumes == volume)][0],
        mode='lines',
        name=f'{volume} A^3'
    ))
s_fig.update_layout(title='Vibrational Entropy vs Temperature', xaxis_title='Temperature (K)', yaxis_title='Entropy (J/K)')
s_fig.show()

In [68]:
# plot the free energy
f_fig = go.Figure()
for i, volume in enumerate(selected_volumes):
    f_fig.add_trace(go.Scatter(
        x=temperatures,
        y=f_vib_v_t[np.where(volumes == volume)][0],
        mode='lines',
        name=f'{volume} A^3'
    ))
f_fig.update_layout(
    title='Vibrational Free Energy vs Temperature',
    xaxis_title='Temperature (K)',
    yaxis_title='Free Energy (eV)'
)
f_fig.show()

In [69]:
# plot the heat capacity
cv_fig = go.Figure()
for i, volume in enumerate(selected_volumes):
    cv_fig.add_trace(go.Scatter(
        x=temperatures,
        y=cv_vib_v_t[np.where(volumes == volume)][0],
        mode='lines',
        name=f'{volume} A^3'
    ))
cv_fig.update_layout(
    title='Vibrational Heat Capacity vs Temperature',
    xaxis_title='Temperature (K)',
    yaxis_title='Heat Capacity (J/K)'
)
cv_fig.show()